# IMPORTING Stock Data

In [ ]:



%matplotlib inline
import datetime
import numpy as np
import pandas as pd
import sklearn
from pandas.io.data import DataReader

def getStockdata(symbol, start_date, end_date):
    
    # Obtain stock information from Yahoo Finance
    ts = DataReader(symbol, "yahoo", start_date-datetime.timedelta(days=365), end_date)

    # Create the new DataFrame
    tsdf = pd.DataFrame(index=ts.index)
    tsdf["Close"] = ts["Adj Close"]
    tsdf["Volume"] = ts["Volume"]
    return tsdf
   
if __name__ == "__main__":
    # Create  series of the Global Stock market Index
    NSDQ = getStockdata("^IXIC", datetime.datetime(2000,1,10), datetime.datetime(2015,11,29))
    print NSDQ 
    SP=getStockdata("^GSPC", datetime.datetime(2000,1,10), datetime.datetime(2015,11,29))
    print SP
    FTSE = getStockdata("^FTSE", datetime.datetime(2000,1,10), datetime.datetime(2015,11,29))
    print FTSE
    NIKKE = getStockdata("^N225", datetime.datetime(2000,1,10), datetime.datetime(2015,11,29))
    print NIKKE
    BSE = getStockdata("^BSESN", datetime.datetime(2000,1,10), datetime.datetime(2015,11,29))
    print BSE
    NSE = getStockdata("^NSEI", datetime.datetime(2000,1,10), datetime.datetime(2015,11,29))
    print NSE
    NSE = getStockdata("^NSEI", datetime.datetime(2000,1,10), datetime.datetime(2015,11,29))
    print NSE
    PSEX = getStockdata("^FCHI", datetime.datetime(2000,1,10), datetime.datetime(2015,11,29))
    print PSEX
    FSEX = getStockdata("^GDAXI", datetime.datetime(2000,1,10), datetime.datetime(2015,11,29))
    print FSEX
    HSEX = getStockdata("^HSI", datetime.datetime(2000,1,10), datetime.datetime(2015,11,29))
    print HSEX

# TIME SERIES ANALYSIS

In [ ]:
get_ipython().magic(u'matplotlib inline')

import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sb
from statsmodels.tsa.stattools import acf  
from statsmodels.tsa.stattools import pacf
import datetime
import numpy as np
import pandas as pd
import sklearn
from pandas.io.data import DataReader


if __name__ == "__main__":
    # Get the stock data in the dataframe
    bse= getStockdata("^BSESN", datetime.datetime(2000,1,10), datetime.datetime(2015,11,29))
    
    sse=getStockdata ("^SSEC", datetime.datetime(2000,1,10), datetime.datetime(2015,11,29))
    
    nsdq = getStockdata("^IXIC", datetime.datetime(2000,1,10), datetime.datetime(2015,11,29))
    
    snp=getStockdata("^GSPC", datetime.datetime(2000,1,10), datetime.datetime(2015,11,29))
   
    ftse = getStockdata("^FTSE", datetime.datetime(2000,1,10), datetime.datetime(2015,11,29))
    
    nikkei = getStockdata("^N225", datetime.datetime(2000,1,10), datetime.datetime(2015,11,29))
    
    nse = getStockdata("^NSEI", datetime.datetime(2000,1,10), datetime.datetime(2015,11,29))
    
    psex = getStockdata("^FCHI", datetime.datetime(2000,1,10), datetime.datetime(2015,11,29))
   
    fsex = getStockdata("^GDAXI", datetime.datetime(2000,1,10), datetime.datetime(2015,11,29))
   
    hsex = getStockdata("^HSI", datetime.datetime(2000,1,10), datetime.datetime(2015,11,29))
   
    
indices=[hsex,sse,bse,nsdq,snp,ftse,nikkei,nse,psex,fsex]

index_name=['HSI','SSE','BSE','NSDQ','S&P500','FTSE','NIKKEI','NSE','FCHI','DAX']
    
def index_analysis(file,x):
        file=file.sort_index()      
        
        print ' '*15,"The analysis for ",index_name[x]        
        ## Plot the closing price of the index
        plt.subplot(221)
        plt.title("Close Price")
        plt.plot(file.index,file["Close"])
       
        
                
        ## plot for the mean with interval of 40 closing values
        moving_average=pd.rolling_mean(file["Close"],40)
        plt.subplot(222)
        plt.title("Moving Average for Close Price")
        plt.plot(file.index,moving_average)
        plt.show()
                

        ## plot for the difference of the present value to the previous value for the closing
        file['First Difference'] = file['Close'] - file['Close'].shift()  
        file['First Difference'].plot()
        
        ##plot for the logged value of the closing price
        file['Log'] = file['Close'].apply(lambda x: np.log(x))  
        file['Log'].plot()
        plt.show()
        
        ## plot for the variance with interval of 40 closing values
        file['Moving Variance'] = pd.rolling_var(file['Close'], 40)  
        file['Log Variance'] = pd.rolling_var(file['Log'], 40)

        fig, ax = plt.subplots(2, 1)  
        file['Moving Variance'].plot(ax=ax[0], title='Moving Variance')  
        file['Log Variance'].plot(ax=ax[1], title='Log Variance')  
        fig.tight_layout()
        
        file['Logged First Difference'] = file['Log'] - file['Log'].shift() 
    
        ##plot for the logged monthly lag    
        file['Monthly Lag'] = file['Logged First Difference'].shift(22)

        sb.jointplot('Logged First Difference', 'Monthly Lag', file, kind='reg')
        plt.show()
   

        lag_correlations = acf(file['Logged First Difference'].iloc[1:])  
        fig, ax = plt.subplots()  
        ax.plot(lag_correlations, marker='o', linestyle='--')
        plt.show()



years=[1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015]

##calculating yearly returns and comparison graphs   
yearly_data=[]
average_yearly_returns=[]
x=0

for i in indices:
    index_analysis(i,x)
    if x<9:
        x+=1
    
for file in indices:
    file=file.sort_index()
    ## Daily percentage change
    file["daily_returns"]=(file["Close"].pct_change())

# Monthly returns
    mon=file["daily_returns"].resample('M',how='sum')
    ret=np.exp(mon)-1
#Yearly returns
    yearly_returns=file["daily_returns"].resample('BA',how='sum')
    year_ret=np.exp(yearly_returns)-1
    
    yearly_data.append(year_ret)
    


plt.figure(1)
plt.subplot(221)
plt.title(index_name[0])
plt.plot(years,yearly_data[0])



plt.subplot(222)
plt.plot(years,yearly_data[1])
plt.title(index_name[1])
plt.show()

plt.subplot(223)
plt.plot(years,yearly_data[2])
plt.title(index_name[2])


plt.subplot(224)
plt.plot(years,yearly_data[3])
plt.title(index_name[3])
plt.show()

plt.figure(1)
plt.subplot(221)
plt.title(index_name[4])
plt.plot(years,yearly_data[4])


plt.subplot(222)
plt.plot(years,yearly_data[5])
plt.title(index_name[5])
plt.show()


#NSE data present only from 2007.
#plt.subplot(223)
#plt.plot(years,yearly_data[6])
#plt.title(index_name[6])


plt.subplot(224)
plt.plot(yearly_data[7])
plt.title(index_name[7])
plt.show()

plt.figure(1)
plt.subplot(221)
plt.title(index_name[8])
plt.plot(years,yearly_data[8])

plt.subplot(222)
plt.title(index_name[9])
plt.plot(years,yearly_data[9])
plt.show()



figr = plt.figure()
ax1 = figr.add_subplot(111)
ax1.patch.set_facecolor('lightgray')

#Plotting the annual percent change on the same plot
ax1.plot(years,yearly_data[0],'r',label=index_name[0])
ax1.plot(years,yearly_data[1],'b',label=index_name[1])
ax1.plot(years,yearly_data[2],'g',label=index_name[2])
ax1.plot(years,yearly_data[3],'y',label=index_name[3])
ax1.plot(years,yearly_data[4],'black',label=index_name[4])
ax1.plot(years,yearly_data[5],'yellow',label=index_name[5])
ax1.plot(years,yearly_data[6],'lime',label=index_name[6])
#ax1.plot(years,yearly_data[7],'darkgreen',label=index_name[7])
ax1.plot(years,yearly_data[8],'violet',label=index_name[8])
ax1.plot(years,yearly_data[9],'pink',label=index_name[8])
ax1.get_xaxis().get_major_formatter().set_useOffset(False)
ax1.set_xlim([2000,2015])
ax1.set_xlabel('YEARS')
ax1.set_ylabel('RETURNS')
ax1.set_title("THE COMPARISON OF RETURNS OF THE STOCK INDICES")

# Shrink current axis by 20%
box = ax1.get_position()
ax1.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
ax1.legend(loc='center left', bbox_to_anchor=(1, 0.5))

plt.show()

fig1 = plt.figure()
ax2 = fig1.add_subplot(111)
ax2.patch.set_facecolor('lightgray')


ax2.plot(years,yearly_data[1],'r',label=index_name[1])
ax2.plot(years,yearly_data[2],'g',label=index_name[2])
ax2.set_xlim([2000,2015])
ax2.get_xaxis().get_major_formatter().set_useOffset(False)
ax2.set_xlabel('YEARS')
ax2.set_ylabel('RETURNS')
ax2.set_title("THE COMPARISON OF RETURNS OF THE STOCK INDICES BSE and SSE")
# Shrink current axis by 20%
box = ax2.get_position()
ax2.set_position([box.x0, box.y0, box.width * 0.8, box.height])

# Put a legend to the right of the current axis
ax2.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()



# BACK TESTING

In [ ]:
import bt
#import getStockdata as data
import datetime
import matplotlib as plt
import pandas as pd
import numpy as np
import seaborn as sb


def BacktestonStocks(df1):
        #Performing Yearly Backtest on the Stock Indexes
        strat1 = bt.Strategy('s1',[bt.algos.RunYearly(),bt.algos.SelectAll(),bt.algos.WeighEqually(),bt.algos.Rebalance()])
        df1=df1.sort_index()
        test = bt.Backtest(strat1,df1)
        res = bt.run(test)
        res.plot()
        res.display()
        res.plot_histogram()
        res.plot_security_weights()


    #Performing Monthly Backtest on the Stock Data
#    strat2 = bt.Strategy('s1',[bt.algos.RunMonthly(),bt.algos.SelectAll(),bt.algos.WeighEqually(),bt.algos.Rebalance()])
#    test2 = bt.Backtest(strat2,df)
#    res2 = bt.run(test2)
#    res2.plot()
#    res2.display()
#    res2.plot_histogram()
#    res2.plot_security_weights()

if __name__ == "__main__":
    # Get Stock Data for various Indexes in the Dataframe
    bse= getStockdata("^BSESN", datetime.datetime(2014,1,10), datetime.datetime(2015,11,29))
    BacktestonStocks(bse)
    sse=getStockdata ("^SSEC", datetime.datetime(2014,1,10), datetime.datetime(2015,11,29))
    BacktestonStocks (sse)
    nsdq = getStockdata("^IXIC", datetime.datetime(2014,1,10), datetime.datetime(2015,11,29))
    BacktestonStocks(nsdq)
    snp=getStockdata("^GSPC", datetime.datetime(2014,1,10), datetime.datetime(2015,11,29))
    BacktestonStocks(snp)
    ftse = getStockdata("^FTSE", datetime.datetime(2014,1,10), datetime.datetime(2015,11,29))
    BacktestonStocks(ftse)
    nikkei = getStockdata("^N225", datetime.datetime(2014,1,10), datetime.datetime(2015,11,29))
    BacktestonStocks(nikkei)
    nse = getStockdata("^NSEI", datetime.datetime(2014,1,10), datetime.datetime(2015,11,29))
    BacktestonStocks(nse)
    psex = getStockdata("^FCHI", datetime.datetime(2014,1,10), datetime.datetime(2015,11,29))
    BacktestonStocks(psex)
    fsex = getStockdata("^GDAXI", datetime.datetime(2014,1,10), datetime.datetime(2015,11,29))
    BacktestonStocks(fsex)
    hsex = getStockdata("^HSI", datetime.datetime(2014,1,10), datetime.datetime(2015,11,29))
    BacktestonStocks(hsex)
    
    

# GARCH MODELLING

In [ ]:
import datetime as dt
import pandas.io.data as web
from arch import arch_model
import pandas as pd

if __name__ == "__main__":
    bse= getStockdata("^BSESN", datetime.datetime(2014,1,10), datetime.datetime(2015,11,29))
    
    sse=getStockdata ("^SSEC", datetime.datetime(2014,1,10), datetime.datetime(2015,11,29))
    
    #test =BacktestonStocks(df)
    #test1=BacktestonStocks(df1)
    nsdq = getStockdata("^IXIC", datetime.datetime(2014,1,10), datetime.datetime(2015,11,29))
    
    snp=getStockdata("^GSPC", datetime.datetime(2014,1,10), datetime.datetime(2015,11,29))
   
    ftse = getStockdata("^FTSE", datetime.datetime(2014,1,10), datetime.datetime(2015,11,29))
    
    nikkei = getStockdata("^N225", datetime.datetime(2014,1,10), datetime.datetime(2015,11,29))
    
    nse = getStockdata("^NSEI", datetime.datetime(2014,1,10), datetime.datetime(2015,11,29))
    
    psex = getStockdata("^FCHI", datetime.datetime(2014,1,10), datetime.datetime(2015,11,29))
   
    fsex = getStockdata("^GDAXI", datetime.datetime(2014,1,10), datetime.datetime(2015,11,29))
   
    hsex = getStockdata("^HSI", datetime.datetime(2014,1,10), datetime.datetime(2015,11,29))
   
    
    indices=[hsex,sse,bse,nsdq,snp,ftse,nikkei,nse,psex,fsex]

index_returns=[]
for i in range (len(indices)):
    data=pd.DataFrame(indices[i])
    #st = dt.datetime(1990,1,1)
    #en = dt.datetime(2014,1,1)
    #data = web.get_data_yahoo('^FTSE', start=st, end=en)
    #df = data
    returns = data['Close'].pct_change().dropna()
    index_returns.append(returns)

    
#figure = index_returns[i].plot()
    
    data['stdev21'] = pd.rolling_std(returns, 21)
    data['hvol21'] = data['stdev21']*(252**0.5) # Annualize.
    data['variance'] = data['hvol21']**2
    data = data.dropna() # Remove rows with blank cells.
    print data.head()
print returns

arch_object=[]
for r in range(len(indices)):
    returns_data=pd.DataFrame(index_returns[r])
    am = arch_model(returns_data) 
    res = am.fit(update_freq=5)
    print(res.summary())

    fig = res.plot(annualize='D')

    am = arch_model(returns_data, p=1, o=1, q=1)
    res = am.fit(update_freq=5, disp='off')
    print(res.summary())

    am = arch_model(returns_data, p=1, o=1, q=1, power=1.0)
    res = am.fit(update_freq=5)
    print(res.summary())

    am = arch_model(returns_data, p=1, o=1, q=1, power=1.0, dist='StudentsT')
    res = am.fit(update_freq=5)
    arch_object.append(res)
    print(res.summary())

##(mu,omega,alpha[1],gamma[1],beta[1],nu)
fixed_res = am.fix([0.00006, 0.0003, 0.00, 0.0958, 0.9322, 4.874])
print(fixed_res.summary())
fixed_res.plot()

df1 = pd.concat([arch_object[0].conditional_volatility,fixed_res.conditional_volatility],1)
df1.columns=['Estimated','Fixed']
df1.plot()


##(mu,omega,alpha[1],gamma[1],beta[1],nu)
fixed_res1 = am.fix([0.00089, 0.0003, 0.0845, 0.0004, 0.9153, 5.0415])
print(fixed_res1.summary())
fixed_res1.plot()

df2 = pd.concat([arch_object[1].conditional_volatility,fixed_res1.conditional_volatility],1)
df2.columns=['Estimated','Fixed']
df2.plot()

fixed_res2 = am.fix([0.0004, 0.0003, 0.000, 0.1018, 0.9246, 6.98])
print(fixed_res2.summary())
fixed_res2.plot()

df3 = pd.concat([arch_object[2].conditional_volatility,fixed_res2.conditional_volatility],1)
df3.columns=['Estimated','Fixed']
df3.plot()

fixed_res3 = am.fix([0.0007, 0.0007, 0.000, 0.2475, 0.8135, 9.0187])
print(fixed_res3.summary())
fixed_res3.plot()

df4 = pd.concat([arch_object[3].conditional_volatility,fixed_res3.conditional_volatility],1)
df4.columns=['Estimated','Fixed']
df4.plot()

fixed_res4 = am.fix([0.0005, 0.0006, 0.000, 0.3094, 0.7880, 12.4586])
print(fixed_res4.summary())
fixed_res4.plot()

df5 = pd.concat([arch_object[4].conditional_volatility,fixed_res4.conditional_volatility],1)
df5.columns=['Estimated','Fixed']
df5.plot()

fixed_res5 = am.fix([-10.1575, 0.00003, 0.0014, -0.0015, 0.9989, 13.0518])
print(fixed_res5.summary())
fixed_res5.plot()

df6 = pd.concat([arch_object[5].conditional_volatility,fixed_res5.conditional_volatility],1)
df6.columns=['Estimated','Fixed']
df6.plot()

fixed_res6 = am.fix([0.0009, 0.0006, 0.0276, 0.2058, 0.8596, 5.7863])
print(fixed_res6.summary())
fixed_res6.plot()

df7 = pd.concat([arch_object[6].conditional_volatility,fixed_res6.conditional_volatility],1)
df7.columns=['Estimated','Fixed']
df7.plot()

fixed_res7 = am.fix([0.0004, 0.0003, 0.000, 0.0920, 0.9312, 6.0092])
print(fixed_res7.summary())
fixed_res7.plot()

df8 = pd.concat([arch_object[7].conditional_volatility,fixed_res7.conditional_volatility],1)
df8.columns=['Estimated','Fixed']
df8.plot()

fixed_res8 = am.fix([-3.2097, 0.085, 1.000, -1.00, 0.5, 2.8462])
print(fixed_res8.summary())
fixed_res8.plot()

df9 = pd.concat([arch_object[8].conditional_volatility,fixed_res8.conditional_volatility],1)
df9.columns=['Estimated','Fixed']
df9.plot()

fixed_res9 = am.fix([-7.2981, 0.0851, 0.4784, -0.4784, 0.7600, 31.558])
print(fixed_res9.summary())
fixed_res9.plot()

df0 = pd.concat([arch_object[9].conditional_volatility,fixed_res9.conditional_volatility],1)
df0.columns=['Estimated','Fixed']
df0.plot()